In [1]:
workspace()

In [2]:
mutable struct RBC{T <: Real}
    α::Float64
    β::Float64
    z::Matrix{T}
    T::Matrix{T}
    K::Float64
    out::Float64
    cs::Float64
    k_grid::Vector{T}
    output::Matrix{Float64}
    f_valueN::Matrix{Float64}
    f_value::Matrix{Float64}
    f_policy::Matrix{Float64}
    n::Int64
    m::Int64

end

In [3]:
function RBC(;α = 1/3,
                   β = 0.95,
                   z = [0.9792 0.9896 1.0000 1.0106 1.0212],
                   T = [0.9727 0.0273 0.0000 0.0000 0.0000;
                        0.0041 0.9806 0.0153 0.0000 0.0000;
                        0.0000 0.0082 0.9837 0.0082 0.0000;
                        0.0000 0.0000 0.0153 0.9806 0.0041;
                        0.0000 0.0000 0.0000 0.0273 0.9727])
    
    K = (α * β)^(1 / (1 - α))
    out = K^α
    cs = out - K
    k_grid = collect(0.5*K:0.00001:1.5*K)
    output = (k_grid.^α) * z;
    n = length(k_grid)
    m = length(z)
    f_valueN = zeros(n, m)
    f_value = zeros(n, m)
    f_policy = zeros(n, m)
    RBC(α, β, z, T, K, out, cs, k_grid, output, f_valueN, f_value, f_policy, n, m)
    
end

RBC

In [4]:
function vfi!(rbc::RBC,
              bellman_operator!::Function; tol::Float64=1e-7,error::Float64=10.0)
        iter = 0
    while error > tol 
        bellman_operator!(rbc)
        error  = maximum(abs.(rbc.f_valueN - rbc.f_value))
        rbc.f_value = rbc.f_valueN
        rbc.f_valueN = zeros(rbc.n, rbc.m)
        iter = iter + 1
         if mod(iter,10)==0 || iter == 1
                println(" Iteration = ", iter, " Sup Diff = ", error)
        end 

    end
        println(" My check = ", rbc.f_policy[1000, 3])
end

vfi! (generic function with 1 method)

In [5]:
function bellman_operator!(rbc::RBC)
       
        E = rbc.f_value * rbc.T'
        for i = 1: rbc.m
            kNext = 1
            for j = 1:rbc.n
                v_max = -1000.0
                k_choice  = rbc.k_grid[1]
                    for l = kNext : rbc.n
                        c = rbc.output[j, i] - rbc.k_grid[l]
                        v = (1 - rbc.β) * log(c) + rbc.β * E[l, i]
                        if v > v_max
                            v_max = v
                            k_choice = rbc.k_grid[l]
                            kNext = l
                        else
                            break 
                        end
                                 
                    end
                    rbc.f_valueN[j, i] = v_max
                    rbc.f_policy[j, i] = k_choice
              end
        end    
     
end


bellman_operator! (generic function with 1 method)

In [6]:
rbc=RBC();

In [7]:
@time vfi!(rbc, bellman_operator!)

 Iteration = 1 Sup Diff = 0.05274159340733661
 Iteration = 10 Sup Diff = 0.031346949265852075
 Iteration = 20 Sup Diff = 0.01870345989335709
 Iteration = 30 Sup Diff = 0.01116551203397076
 Iteration = 40 Sup Diff = 0.00666854170813258
 Iteration = 50 Sup Diff = 0.003984292748717033
 Iteration = 60 Sup Diff = 0.0023813118039327508
 Iteration = 70 Sup Diff = 0.0014236586450983024
 Iteration = 80 Sup Diff = 0.0008513397747205165
 Iteration = 90 Sup Diff = 0.0005092051752288995
 Iteration = 100 Sup Diff = 0.00030462324421465237
 Iteration = 110 Sup Diff = 0.00018226485632300005
 Iteration = 120 Sup Diff = 0.00010906950872624499
 Iteration = 130 Sup Diff = 6.527643736320421e-5
 Iteration = 140 Sup Diff = 3.907108211997912e-5
 Iteration = 150 Sup Diff = 2.3388077119990136e-5
 Iteration = 160 Sup Diff = 1.4008644637186762e-5
 Iteration = 170 Sup Diff = 8.391317202871562e-6
 Iteration = 180 Sup Diff = 5.026474537817016e-6
 Iteration = 190 Sup Diff = 3.010899863653549e-6
 Iteration = 200 Sup Di